In [1]:
%pylab
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
# get the predicted matrix
X = np.loadtxt("data/X.csv", delimiter=",")
Y = np.loadtxt("data/Y.csv", delimiter=",")
P_hat = np.dot(X, Y.T)

In [ ]:
train_data = pd.read_csv("data/train_data.csv")
train_matrix = pd.pivot_table(train_data, values="quantity", index=["customer_id"], columns=["item_id"],
                             aggfunc=np.sum, fill_value=0)
R = train_matrix.values

In [28]:
# C is the confidence matrix
C = np.ones((R.shape[0], R.shape[1]))
C += 40 * R

In [30]:
I_f = np.zeros((3, 3))  # we first let factor dimension to be 3
np.fill_diagonal(I_f, 1)
I_n = np.zeros((R.shape[1], R.shape[1]))
np.fill_diagonal(I_n, 1)

In [31]:
# P is the preference matrix
P = np.zeros((R.shape[0], R.shape[1]))
P[R > 0] = 1

In [74]:
### demostrate for 10th user, recommend 10 items
recommend_list_3 = np.argsort(P_hat[9])[::-1][0:3]
recommended_items = train_matrix.columns[recommend_list_3].values
recommended_items

array(['D1792952280500', 'D1792952280600', 'D1793452400001'], dtype=object)

In [35]:
C_u = np.zeros((R.shape[1], R.shape[1]))
for index in xrange(R.shape[1]):
    C_u[index][index] = C[9][index]
W_u = inv(np.dot(Y.T, Y) + np.dot(np.dot(Y.T, C_u - I_n), Y) + 2 * I_f)  ## 3 * 3 matrix

In [39]:
P_9_nonzero_index = np.nonzero(P[9])[0]

In [41]:
c_u = C[9][P_9_nonzero_index]

In [44]:
s_u = []
p_hat_ui_decomposed_array = []
for i in recommend_list_3:
    s_u.append(np.dot(np.dot(Y[i], W_u), Y.T[:, P_9_nonzero_index]))

for i in xrange(len(recommend_list_3)):
    p_hat_ui_decomposed_array.append(np.multiply(s_u[i], c_u))
p_hat_ui_decomposed_array

[array([ 0.05199538,  0.04514935,  0.04945257, -0.04091673,  0.17245023,
         0.06586432,  0.28739602,  0.07461416,  0.0997171 ]),
 array([ 0.05870066,  0.06651653,  0.14409597,  0.10174654,  0.03674888,
         0.05076268,  0.18769832,  0.09662078,  0.0607922 ]),
 array([ 0.04994686,  0.04516059,  0.06282878, -0.01689346,  0.15804223,
         0.06939592,  0.2602177 ,  0.07614116,  0.09055445])]

In [50]:
p_hat_ui_decomposed_array_argsort = np.argsort(p_hat_ui_decomposed_array)

In [54]:
p_hat_ui_decomposed_array_argsort

array([[3, 1, 2, 0, 5, 7, 8, 4, 6],
       [4, 5, 0, 8, 1, 7, 3, 2, 6],
       [3, 1, 0, 2, 5, 7, 8, 4, 6]])

In [62]:
p_hat_reverse = []
for i in xrange(len(recommend_list_3)):
    p_hat_reverse.append(p_hat_ui_decomposed_array_argsort[i][::-1])

In [65]:
p_hat_reverse = np.delete(p_hat_reverse, np.arange(3, len(p_hat_reverse[0])), 1)

In [85]:
P_9_nonzero_index[p_hat_reverse]

array([[1404, 1233, 2038],
       [1404,  734,  879],
       [1404, 1233, 2038]])

In [86]:
train_matrix.columns[P_9_nonzero_index[p_hat_reverse]]

Index([[u'D1793452400001', u'D1777551370990', u'P1E46123680500'],
       [u'D1793452400001', u'D1540784170001', u'D1557346410250'],
       [u'D1793452400001', u'D1777551370990', u'P1E46123680500']],
      dtype='object', name=u'item_id')

In [88]:
p_hat_ui_decomposed_array[0][p_hat_reverse[0]]

array([ 0.28739602,  0.17245023,  0.0997171 ])

In [95]:
for i in xrange(len(p_hat_reverse[0])):
    print "recommend: %s, predicted preference: %f" % (recommended_items[i], np.sum(p_hat_ui_decomposed_array[i]))
    print "purchased: %s" % (train_matrix.columns.values[P_9_nonzero_index[p_hat_reverse[i]]])
    print "contribution: %s" % (p_hat_ui_decomposed_array[i][p_hat_reverse[i]])
    print "Ratio: %s\n" % np.true_divide(np.sum(p_hat_ui_decomposed_array[i][p_hat_reverse[i]]), np.sum(p_hat_ui_decomposed_array[i]))

recommend: D1792952280500, predicted preference: 0.805722
purchased: ['D1793452400001' 'D1777551370990' 'P1E46123680500']
contribution: [ 0.28739602  0.17245023  0.0997171 ]
Ratio: 0.694486514562

recommend: D1792952280600, predicted preference: 0.803683
purchased: ['D1793452400001' 'D1540784170001' 'D1557346410250']
contribution: [ 0.18769832  0.14409597  0.10174654]
Ratio: 0.539442880696

recommend: D1793452400001, predicted preference: 0.795394
purchased: ['D1793452400001' 'D1777551370990' 'P1E46123680500']
contribution: [ 0.2602177   0.15804223  0.09055445]
Ratio: 0.639700870377



In [96]:
P_9_nonzero_index[p_hat_reverse[0]]

array([1404, 1233, 2038])

In [103]:
train_matrix.iloc[9, P_9_nonzero_index[p_hat_reverse[1]]]

item_id
D1793452400001    1
D1540784170001    1
D1557346410250    1
Name: 000004_LAP, dtype: int64

In [105]:
train_matrix.iloc[9, P_9_nonzero_index[p_hat_reverse[2]]]

item_id
D1793452400001    1
D1777551370990    1
P1E46123680500    1
Name: 000004_LAP, dtype: int64